# **Mental Health LLM Project**

## **Importing Libararies**

In [1]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import ConversationalRetrievalChain
from langchain.llms import Ollama
import warnings
warnings.filterwarnings('ignore')

## **Data Reading**

In [2]:
pdf_paths = [
    "9241562579.pdf",
    "wcms_108221.pdf",
    "youth-mental-health.pdf"
]

all_documents = []

for pdf_path in pdf_paths:
    loader = PyPDFLoader(pdf_path)
    documents = loader.load()
    all_documents.extend(documents)

## **Data Splliting into Chunks**

In [3]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
    separators=["\n\n", "\n", ".", " "]
)

texts = text_splitter.split_documents(documents)

## **Embedding Data**

In [4]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

## **Vector Store DB**

In [5]:
vector_store = FAISS.from_documents(texts, embeddings)

## **Set up Ollama LLM for conversation**

In [8]:
chat_model = Ollama(model="llama3.2")

retrieval_chain = ConversationalRetrievalChain.from_llm(
    llm=chat_model,
    retriever=vector_store.as_retriever(),
    return_source_documents=True
)

## **Chatbot interaction**

In [10]:
chat_history = []

while True:
    user_query = input("You: ")
    if user_query.lower() == "exit":
        print("Goodbye!")
        break

    response = retrieval_chain({
        "question": user_query,
        "chat_history": chat_history
    })
    
    chat_history.append((user_query, response['answer']))

    print(f"Answer: {response['answer']}")

    break

Answer: I don't know the answer to this question based on the provided context. The text discusses mental health conditions in general, particularly among young people, but does not differentiate between "mental health" and "youth mental health".
